# Example: Divergence method

In [ ]:
import os

import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ucat
import xarray as xr

import ddeq

## SMARTCARB dataset
A brief example demonstrating the application of the divergence method to estimate CO2 and NO2 emissions of the Boxberg power plant in the SMARTCARB dataset. The example requires both the SMARTCARB Level-2 data and the wind fields that you can download here: https://doi.org/10.5281/zenodo.4048227. To run the example, you need the download and extract the CO2M dataset (`Sentinel_CO2.tar.gz` file) and the SMARTCARB wind fields (`cosmo2D_2015-01.tar.gz` and `cosmo2D_2015-07.tar.gz`).

In [ ]:
# modify to point to your SMARTCARB dataset
SMARTCARB_DATA_PATH = "/output/SMARTCARB/output/level2/Sentinel_7_CO2"
SMARTCARB_WIND_PATTERN = 'SMARTCARB_winds_%Y%m%d%H.nc'
SMARTCARB_WIND_PATH = "/output/COSMO-GHG/SMARTCARB/"

In [ ]:
sources = ddeq.misc.read_point_sources()
sources = sources.sel(source=['Boxberg'])

l2 = ddeq.smartcarb.Level2Dataset(SMARTCARB_DATA_PATH, 'ace', co2_noise_scenario='low', no2_noise_scenario='low',
                                  make_no2_error_cloud_dependent=False)

res = ddeq.div.estimate_emissions(
    l2, SMARTCARB_WIND_PATH, sources,
    wind_product='SMARTCARB', pattern=SMARTCARB_WIND_PATTERN,
    trace_gases=['CO2', 'NO2'],
    varnames=['CO2', 'NO2'],
    smooth_data=[True, False],
    remove_background=[True, False],
    start_date='2015-02-01', end_date='2015-05-01')

In [ ]:
res

The estimated emissions of Boxberg are 760.2$\pm$61.56 kg/s and 0.5766$\pm$0.06649 kg/s for CO2 and NO2 using low-noise CO2 and NO2 for constellation ace from 1 Feb to 1 May 2015.

### Compare with true emissions

In [ ]:
times = pd.date_range('2015-02-01', '2015-05-01')
co2_true = np.mean([ddeq.smartcarb.read_true_emissions('CO2', 'Boxberg', t, units="Mt/a").mean() for t in times])
no2_true = np.mean([ddeq.smartcarb.read_true_emissions('NO2', 'Boxberg', t, units="kt/a").mean() for t in times])

In [ ]:
co2_est = ucat.convert_mass_per_time_unit(res.sel(source='Boxberg')['CO2_emissions'], "kg/s", "Mt/a")
co2_pre = ucat.convert_mass_per_time_unit(res.sel(source='Boxberg')['CO2_emissions_precision'], "kg/s", "Mt/a")

no2_est = ucat.convert_mass_per_time_unit(res.sel(source='Boxberg')['NO2_emissions'], "kg/s", "kt/a")
no2_pre = ucat.convert_mass_per_time_unit(res.sel(source='Boxberg')['NO2_emissions_precision'], "kg/s", "kt/a")

print(f'CO2 estimate: {co2_est:.1f}±{co2_pre:.1f} Mt/a, True: {co2_true:.1f} Mt/a')
print(f'NO2 estimate: {no2_est:.1f}±{no2_pre:.1f} kt/a, True: {no2_true:.1f} kt/a')

## TROPOMI NO2 product
This example uses TROPOMI NO2 images and ERA-5 wind fields prepared (but not published) in the CoCO2 project. To apply the divergence method to TROPOMI data, TROPOMI data can be downloaded and prepared using *ddeq* (see: `example_download_tropomi.ipynb`). 

In [ ]:
# this example uses data prepare in the CoCO2 project
DATA_PATH = '/input/CORSO/TROPOMI/Arnot/NO2'
WIND_PATH = '/input/ERA5/CORSO/gnfra/'

In [ ]:
sources = ddeq.misc.read_point_sources()
sources = sources.sel(source=['Matimba'])

In [ ]:
s5p_l2 = ddeq.sats.Level2TropomiDataset('Arnot_S5P_????_L2__NO2____%Y%m%dT????*.nc', root=DATA_PATH)

In [ ]:
res = ddeq.div.estimate_emissions(
    s5p_l2, WIND_PATH, sources,
    wind_product='ERA5', pattern='ERA5-gnfra-%Y%m%dt%H00_SouthernAfrica.nc',
    trace_gases=['NO2'],
    #start_date='2021-01-01', end_date='2021-01-31', hour=12,
    start_date='2021-01-01', end_date='2021-12-31', hour=12,
    varnames=['nitrogendioxide_tropospheric_column'],
    smooth_data=[False],
    remove_background=[False]
)

In [ ]:
DOMAIN = ddeq.misc.Domain('Matimba', 27, -24.2, 28.2, -23.1)

amf_correction = 1.27
f = 2.5

fig, axes = plt.subplots(1,2, figsize=(9.5,3.3),
                         subplot_kw={'projection': ccrs.PlateCarree()})

this = res.sel(source='Matimba')

fig, ax, _ = ddeq.vis.create_map(DOMAIN, ax=axes[0], admin_level=1, edgecolor='k')
m = ax.pcolormesh(
    this.longrid, this.latgrid,
    amf_correction * ucat.convert_columns(this['NO2_grid'], 'm-2', 'umol m-2', molar_mass='NO2'),
    vmin=0, vmax=300, transform=ccrs.PlateCarree()
)
plt.colorbar(m, ax=ax).set_label('NO$_2$ enhancement [µmol m$^{-2}$]')

fig, ax, _ = ddeq.vis.create_map(DOMAIN, ax=axes[1], admin_level=1, edgecolor='k')
m = ax.pcolormesh(
    this.longrid, this.latgrid,
    f * amf_correction * this['NO2_div'],
    vmin=-2, vmax=6, transform=ccrs.PlateCarree()
)
plt.colorbar(m, ax=ax).set_label('NO$_x$ flux [g m$^{-2}$ s$^{-1}$]')

for i, ax in enumerate(axes.flatten()):
    ddeq.vis.add_gridlines(ax, dlon=0.2, dlat=0.2)
    ddeq.vis.add_hot_spots(
        ax, sources=sources,
        do_path_effect=True, ms=3, size='small'
    )
    ax.set_title(f'({"abcd"[i]})')


em = amf_correction * f * ucat.convert_mass_per_time_unit(res['NO2_emissions'][0], "kg/s", "kt/a")
em_std = amf_correction * f * ucat.convert_mass_per_time_unit(res['NO2_emissions_precision'][0], "kg/s", "kt/a")

axes[1].set_title(f'(b) Q = {em:.1f} kt NO$_2$ a$^{{-1}}$')

plt.tight_layout()